In [1]:
from dotenv import load_dotenv
import openai
import os
from openai import AzureOpenAI
from IPython.display import display, HTML, JSON, Markdown, Image

load_dotenv()
AZURE_OPENAI_ENDPOINT=os.getenv("AISTUDIO_AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_GPT4o_DEPLOYMENT=os.getenv("AI_STUDIO_AZURE_OPENAI_GPT4o_DEPLOYMENT")
AZURE_OPENAI_API_VERSION="2024-02-01"
AZURE_OPENAI_KEY=os.getenv("AISTUDIO_AZURE_OPENAI_KEY")
 
AZURE_COMPUTER_VISION_ENDPOINT = os.getenv("AZURE_COMPUTER_VISION_ENDPOINT")
AZURE_COMPUTER_VISION_KEY = os.getenv("AZURE_COMPUTER_VISION_KEY")

client = openai.AzureOpenAI(
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_KEY,
        api_version=AZURE_OPENAI_API_VERSION
)


In [2]:
import os
import json
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential

def analyze_image(image_filename):
    client = ImageAnalysisClient(
        endpoint=AZURE_COMPUTER_VISION_ENDPOINT,
        credential=AzureKeyCredential(AZURE_COMPUTER_VISION_KEY)
    )

    with open(image_filename, "rb") as image_stream:
        result = client.analyze(
            image_stream,
            visual_features=[VisualFeatures.CAPTION, VisualFeatures.READ, VisualFeatures.DENSE_CAPTIONS, VisualFeatures.OBJECTS, VisualFeatures.PEOPLE, VisualFeatures.TAGS, VisualFeatures.SMART_CROPS],
            gender_neutral_caption=True,  # Optional (default is False)
        )

    print("Image analysis results:")
    print(" Model version: {}".format(result.model_version))
    # Print caption results to the console
    if result.caption is not None:
        print(" Caption:")    
        print(f"   '{result.caption.text}', Confidence {result.caption.confidence:.4f}")
    if result.dense_captions is not None:
        print(" Dense Captions:")    
        for caption in result.dense_captions.list:
            print(f"   Text: '{caption.text}', Confidence: {caption.confidence}")
    if result.objects is not None:
        print(" Objects:")    
        for caption in result.objects.list:
            print(f"   Text: '{caption.text}', Confidence: {caption.confidence}")
    if result.tags is not None:
        print(" Tags:")
        for caption in result.tags.list:
            print(f"   Text: '{caption.name}', Confidence: {caption.confidence}")
    if result.people is not None:
        print(" People:")    
        print(f"   '{result.people.list}'")
    if result.smart_crops is not None:
        print(" Smart Crops:")    
        print(f"   '{result.smart_crops.list}'")
    
    result_dict = result.as_dict()
    result_json = json.dumps(result_dict)
    return result_json
            


#### Analyze a picture using Azure Cognitve services to extract text from a picture

In [3]:
image_url = "./data/retail/bike.jpg"
json_result = analyze_image(image_url)

Image analysis results:
 Model version: 2023-10-01
 Caption:
   'a blue and pink polo shirt', Confidence 0.7037
 Dense Captions:
   Text: 'a blue and pink polo shirt', Confidence: 0.7118337750434875
   Text: 'a blue shirt with purple and pink design', Confidence: 0.6390741467475891
   Text: 'a blue shirt with a tie', Confidence: 0.6828346848487854
 Objects:
 Tags:
   Text: 'clothing', Confidence: 0.999369740486145
   Text: 'fabric', Confidence: 0.9067331552505493
   Text: 'sleeve', Confidence: 0.9034119844436646
   Text: 'collar', Confidence: 0.8999518156051636
   Text: 'active shirt', Confidence: 0.8928235769271851
   Text: 'top', Confidence: 0.892290472984314
   Text: 'shirt', Confidence: 0.813533365726471
   Text: 'person', Confidence: 0.6144185662269592
 People:
   '[{'boundingBox': {'x': 103, 'y': 0, 'w': 113, 'h': 213}, 'confidence': 0.0026073618791997433}, {'boundingBox': {'x': 0, 'y': 0, 'w': 63, 'h': 223}, 'confidence': 0.0021513751707971096}, {'boundingBox': {'x': 0, 'y': 0, 

#### Extract all dense captions from the json result

In [4]:
import json
dict = json.loads(json_result)
dense_captions = dict["denseCaptionsResult"]["values"]


#### Generate a product description from the text extracted from the photo using OpenAI

In [10]:
def call_openAI(system, text):
    message_text = [
        {"role":"system","content":system},
        {"role":"user","content":text}
    ]

    response = client.chat.completions.create(
        model=AZURE_OPENAI_GPT4o_DEPLOYMENT,
        messages = message_text,
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    return response.choices[0].message.content

In [11]:
system ="""
         You are an AI assistant that generates product descriptions for an online retail website especializing in clothing.
        Given the user's input, generate a Product Description.
"""
count = 1
for caption in dense_captions:
    result = call_openAI(system, caption["text"])
    print("Option " + str(count) + ": " + result)
    print()
    count = count + 1


Option 1: Introducing our stylish Blue and Pink Polo Shirt, a perfect blend of comfort and contemporary fashion. Crafted from high-quality, breathable cotton fabric, this polo shirt ensures you stay cool and comfortable all day long. The classic design features a vibrant blue base accented with playful pink stripes, adding a touch of modern flair to a timeless look.

Complete with a ribbed collar and a three-button placket, this polo shirt offers a polished appearance suitable for both casual outings and semi-formal occasions. The relaxed fit allows for easy movement, making it an ideal choice for everything from weekend brunches to a day at the golf course.

Pair it with your favorite jeans or chinos for an effortlessly stylish ensemble. Elevate your wardrobe with this versatile Blue and Pink Polo Shirt—where comfort meets chic.

Option 2: Elevate your wardrobe with this stylish blue shirt featuring an eye-catching purple and pink design. Perfect for both casual and semi-formal occasi

In [12]:
#translate to spanish
system ="""
         You are an AI assistant that generates product descriptions in spanish for an online retail website especializing in clothing.
        Given the user's input, generate a Product Description.
"""
count = 1
for caption in dense_captions:
    result = call_openAI(system, caption["text"])
    print("Option " + str(count) + ": " + result)
    print()
    count = count + 1

Option 1: Descubre nuestra elegante camisa polo en tonos azul y rosa, una prenda imprescindible para cualquier guardarropa moderno. Confeccionada con materiales de alta calidad, esta polo ofrece una comodidad excepcional y un estilo impecable. El diseño en dos tonos, azul y rosa, añade un toque de frescura y originalidad, perfecto para cualquier ocasión, ya sea una reunión casual o una salida con amigos. Su corte clásico y versátil se adapta a todas las siluetas, garantizando un ajuste perfecto. ¡Añade un toque de color y estilo a tu colección con esta magnífica camisa polo!

Option 2: **Camiseta Azul con Diseño en Púrpura y Rosa**

Añade un toque de estilo y color a tu guardarropa con esta camiseta azul impresionante, adornada con un diseño vibrante en púrpura y rosa. Perfecta para cualquier ocasión, esta camiseta combina comodidad y moda de una manera única. Confeccionada con materiales de alta calidad, garantiza durabilidad y una sensación suave en la piel. El contraste entre el azu